### Import packages

In [1]:
# Scientific computing
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# Import the storage tank Class
from cryoevap.storage_tanks import Tank

# Import Cryogen class
from cryoevap.cryogens import Cryogen

### Define all Variables

In [ ]:
# Cylindrical storage tank properties
Q_roof = 0 # Roof heat ingress / W
T_air = 293.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 0.1 # W/m2/K
U_V = 0.1 # W/m2/K

# Specify wall heat partitioning fraction
eta_w= 0.9

# Specify tank operating pressure
P = 101325 # Pa

### Variables of interest

# Vertically orientated cylindrical tank volume
V_tank = 0.007 #m^3 | values of 80, 60000 and 165000

# Initial liquid filling / Dimensionless
LF = 0.95 # values of 0.95, 0.80 and 0.30

# Specify heat transfer flux at the bottom
# This will represent the heat conduction from the piping system

q_b = 0 # W/m^2 13.09, 1.99, 0

# especify compound in the tank
Compound = "nitrogen" # hydrogen, methane, ammonia, nitrogen


In [ ]:
# Define evaporation time to simulate
hours = 24
evap_time = 3600 * hours

# Thickness of the in % of the internal diameter
ST = 1.02

# Show sim duration
print(f"Simulation time: {hours} h")

In [ ]:
from scipy.optimize import Bounds, minimize

# Minimum and maximum practical ranges
# of the aspect ratio
bounds = Bounds([0.1], [10])

# Define objective function
def BOR_function(a):
    # Update aspect ratio
    # Calculate internal diameter
    d_i = ((4 * V_tank)/(np.pi * a))**(1/3) # internal diameter / m
    d_o = d_i * ST # external diameter / m
    
    # Initialize tank
    mid_tank = Tank(d_i, d_o, V_tank, LF)

    Q_b=q_b*mid_tank.A_T
    
    mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)

    mid_tank.U_roof = U_V
    
    # Set cryogen
    mid_tank.cryogen = Cryogen(name = Compound)
    
    mid_tank.cryogen.set_coolprops(P)
    
    # Define vertical spacing
    dz = 0.003

    # Calculate number of nodes
    
    n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

    # Define dimensionless computational grid
    mid_tank.z_grid = np.linspace(0, 1, n_z)
    print(len(mid_tank.z_grid))
    # Execute simulation
    mid_tank.evaporate(evap_time)
    
    # Calculate BOR
    BOR = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]

    print("a = %.3f, BOR=%.3e" % (a, BOR))
    
    return BOR

# Initial liquid filling to optimise

x0 = 1 
res = minimize(BOR_function, x0, method='trust-constr', options={'verbose': 1}, bounds=bounds)

if res.success:
   print(f"\nOptimum found for Tank Volume: {V_tank}, Liquid Filling: {LF*100}%, Compound: {Compound}" + f"\nAspect Ratio = {res.x[0]:.3f}" + f"\nBOR = {res.fun*100:.3f} %")

In [ ]:
#Boil-off ratio list for plotting
BOR_plot = []

# Generate aspect ratio range
AR = np.linspace(1.3,1.7,30)
# Generate tanks in Aspect Ratio range
for a in AR:
    # Update aspect ratio
    # Calculate internal diameter
    d_i = ((4 * V_tank)/(np.pi * a))**(1/3) # internal diameter / m
    d_o = d_i * ST # external diameter / m
    
    # Initialize tank
    mid_tank = Tank(d_i, d_o, V_tank, LF)

    Q_b=q_b * mid_tank.A_T
    
    mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_b, Q_roof, eta_w)

    mid_tank.U_roof = U_V
    
    # Set cryogen
    mid_tank.cryogen = Cryogen(name = Compound)
    
    mid_tank.cryogen.set_coolprops(P)
    
    # Define vertical spacing
    dz = 0.1

    # Calculate number of nodes
    n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

    # Define dimensionless computational grid
    mid_tank.z_grid = np.linspace(0, 1, n_z)
    print(len(mid_tank.z_grid))
    # Execute simulation
    mid_tank.evaporate(evap_time)
    
    # Calculate BOR
    BOR = 1 - mid_tank.data['V_L'][-1] / mid_tank.data['V_L'][0]
    
    # Save BOR value
    BOR_plot.append(BOR*100)
    
    print("a = %.3f, BOR=%.3e" % (a, BOR))
    # print("BOR = %.3f %% | B_l = %.3f " % (BOR * 100, mid_tank.b_l_dot))
    # print(f"Aspect Ratio:{a:.3f}, Altura: {mid_tank.l:.3f} , Diametro interno:{mid_tank.d_i:.3f}, Volumen:{mid_tank.V} \n")
print("Simulation end")

In [ ]:
# Create a plot
plt.plot(AR, BOR_plot,"ro")

# Add labels and title
plt.xlabel('Aspect Ratio | a')
plt.ylabel('Boil-off Ratio | %')
plt.title(f'Tank Volume: {V_tank}, Liquid Filling: {LF*100}%, Compound: {Compound}')


# Save the plot
plt.savefig(f'{Compound}/V_{V_tank}_LF_{LF*100}_C_{Compound}_evap.png')

# Display the plot
plt.show()